<a href="https://colab.research.google.com/github/gabrielmahia/AI-KungFU/blob/master/ModifiedAllStock_TradeBot_Test4_LSTM_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

def get_stock_data(ticker):
    try:
        end_date = datetime.datetime.now() - datetime.timedelta(days=1)
        stock_data = yf.download(ticker, start="2010-01-01", end=end_date.strftime("%Y-%m-%d"))
        if stock_data.empty:
            print("No data found for the given ticker. Please try again.")
            return None
        return stock_data
    except Exception as e:
        print("Error fetching data. Please try again.")
        return None

def create_dataset(data, look_back=60):
    X, y = [], []
    for i in range(look_back, len(data)):
        X.append(data[i - look_back:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

def split_data(data):
    train_data, temp_data = train_test_split(data, train_size=0.7, shuffle=False)
    validation_data, test_data = train_test_split(temp_data, train_size=0.5, shuffle=False)
    return train_data, validation_data, test_data

def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def train_and_evaluate(look_back, X_train, y_train, X_validation, y_validation):
    X_train_lb, y_train_lb = create_dataset(X_train, look_back)
    X_validation_lb, y_validation_lb = create_dataset(X_validation, look_back)

    X_train_lb = np.reshape(X_train_lb, (X_train_lb.shape[0], X_train_lb.shape[1], 1))
    X_validation_lb = np.reshape(X_validation_lb, (X_validation_lb.shape[0], X_validation_lb.shape[1], 1))

    model = create_model((X_train_lb.shape[1], 1))
    model.fit(X_train_lb, y_train_lb, epochs=5, batch_size=64, verbose=0)

    validation_pred = model.predict(X_validation_lb)
    mae = mean_absolute_error(y_validation_lb, validation_pred)

    return mae, model

while True:
    ticker = input("\nPlease enter the stock ticker (e.g., AAPL for Apple Inc.) or type 'quit' to exit: ").upper()
    if ticker == 'QUIT':
        break
    stock_data = get_stock_data(ticker)
    if stock_data is None:
        continue

    # Prepare data
    data = stock_data['Close'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Create training, validation, and testing datasets
    train_data, validation_data, test_data = split_data(scaled_data)

    # Split data into features and targets
    X_train, y_train = train_data, train_data
    X_validation, y_validation = validation_data, validation_data

    # Find the best look-back period
    look_back_range = range(30, 121, 10)
    best_mae = float('inf')
    best_look_back = None
    best_model = None

    for look_back in look_back_range:
        mae, model = train_and_evaluate(look_back, X_train, y_train, X_validation, y_validation)

        if mae < best_mae:
            best_mae = mae
            best_look_back = look_back
            best_model = model

    # Prepare the test data using the best look-back period
    X_train_lb, y_train_lb = create_dataset(train_data, best_look_back)
    X_test_lb, y_test_lb = create_dataset(test_data, best_look_back)
    X_train_lb = np.reshape(X_train_lb, (X_train_lb.shape[0], X_train_lb.shape[1], 1))
    X_test_lb = np.reshape(X_test_lb, (X_test_lb.shape[0], X_test_lb.shape[1], 1))

    # Make predictions
    train_pred = best_model.predict(X_train_lb)
    test_pred = best_model.predict(X_test_lb)

    # Invert the scaling to get the original stock prices
    train_pred = scaler.inverse_transform(train_pred)
    test_pred = scaler.inverse_transform(test_pred)

    # Predict the next 5 days
    future_data = scaled_data[-best_look_back:]
    future_data = np.reshape(future_data, (1, best_look_back, 1))
    future_pred = best_model.predict(future_data)
    future_pred = scaler.inverse_transform(future_pred)

    print(f"\nPredictions for {ticker} for the next 5 days:")
    last_date = datetime.datetime.now()
    for i in range(1, 6):
        next_date = last_date + datetime.timedelta(days=i)
        print(f"{next_date.strftime('%Y-%m-%d')}: {future_pred[0][0]:.2f}")






Please enter the stock ticker (e.g., AAPL for Apple Inc.) or type 'quit' to exit: spy
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 24ms/step

Predictions for SPY for the next 5 days:
2023-04-01: 394.78
2023-04-02: 394.78
2023-04-03: 394.78
2023-04-04: 394.78
2023-04-05: 394.78

Please enter the stock ticker (e.g., AAPL for Apple Inc.) or type 'quit' to exit: quit
